In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import six
from six.moves import range

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
X = np.loadtxt('iris.data', dtype='object', delimiter=',')
Y = X[:,-1]
X = X[:, :-1].astype('f')
X.shape, Y.shape, Y.dtype

((150, 4), (150,), dtype('O'))

`X` is a `NxK` float matrix where each row (`X[i]`) corresponds to a data point.

In [4]:
def gmm(X, n_classes, n_iter):
    
    def init(X):
        _, n = X.shape
        return np.random.rand(n_classes, n), 2 * np.random.rand(n_classes, n, n) + 1, np.random.rand(n_classes)
    
    
    mean, cov, alpha = init(X)

    def prob(x, mean, cov):
        n = len(x)
        dim = np.shape(cov)[0]
        
        covdet = np.linalg.det(cov + np.eye(dim) * 0.001)
        covinv = np.linalg.inv(cov + np.eye(dim) * 0.001)
        xdiff = (x - mean).reshape((1,dim))
        
        prob = 1.0/(np.power(np.power(2*np.pi,dim)*np.abs(covdet),0.5))*\
               np.exp(-0.5*xdiff.dot(covinv).dot(xdiff.T))[0][0]
        return prob
        
    
    # EM algorithm
    
    mat = np.zeros((len(X), n_classes))
    for times in range(n_iter):
        for j, x in enumerate(X):
            temp, tempP = 0, 0
            for i in range(n_classes):
                tempP = prob(x, mean[i], cov[i])
                temp += tempP
                mat[j][i] = alpha[i] * tempP
            mat[j] /= temp
        
        for i in range(n_classes):
            # updata mean
            mean[i] = np.dot(mat[:, i].T, X) / sum(mat[:, i])
            
            # update cov
            temp = np.zeros(cov[0].shape)
            for j in range(len(X)):
                data = (X[j] - mean[i]).reshape(4, 1)
                temp += mat[j][i] * np.dot(data, data.T)
            temp /= sum(mat[:, i])
            cov[i] = temp
            alpha[i] = sum(mat[:, i]) / len(X)
            
    
    class_assignments = np.zeros(len(X))
    for j, x in enumerate(X):
        temp, tempP = 0, 0
        for i in range(n_classes):
            tempP = prob(x, mean[i], cov[i])
            temp += tempP
            mat[j][i] = alpha[i] * tempP
        mat[j] /= temp
        class_assignments[j] = np.argmax(mat[j])
    return class_assignments,mean,cov

In [ ]:
class_assignments, mean, cov = gmm(X, 3, 600)  # You may want to tune the number of iterations

## Visualization: a Cross Section

In [ ]:
plt.figure(figsize=(9,4))
plt.subplot(121)
for k in range(3):
    plt.scatter(X[class_assignments==k,2], X[class_assignments==k, 1], s=2)
plt.subplot(122)
for k, class_name in enumerate(np.unique(Y)):
    plt.scatter(X[Y==class_name, 2], X[Y==class_name, 1], s=2)

## Visualization: PCA Projection

In [ ]:
evals, evecs = np.linalg.eigh(np.cov(X.T))
to_crd = lambda x: ((x-x.mean(axis=0))@evecs)[:,-2:]
crds = to_crd(X)

In [ ]:
plt.figure(figsize=(9,4))
plt.subplot(121)
for k in range(3):
    plt.scatter(crds[class_assignments==k, 0], crds[class_assignments==k, 1], s=2)
plt.scatter(to_crd(mean)[:,0], to_crd(mean)[:,1], s=30, marker='+')
plt.subplot(122)
for k in np.unique(Y):
    plt.scatter(crds[Y==k, 0], crds[Y==k, 1], s=2)